Develop CovidDB MIS application:
Covid-Data-Global :
1. To find out the death percentage locally and globally

In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MakerPlan").getOrCreate()
df = spark.read.csv('worldometer.csv',header=True,inferSchema=True)
df.show()
df.printSchema()

+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|Country/Region|    Continent|Population|TotalCases|NewCases|TotalDeaths|NewDeaths|TotalRecovered|NewRecovered|ActiveCases|Serious,Critical|Tot Cases/1M pop|Deaths/1M pop|TotalTests|Tests/1M pop|          WHO Region|
+--------------+-------------+----------+----------+--------+-----------+---------+--------------+------------+-----------+----------------+----------------+-------------+----------+------------+--------------------+
|           USA|North America| 331198130|   5032179|    NULL|     162804|     NULL|       2576668|        NULL|    2292707|           18296|           15194|        492.0|  63139605|      190640|            Americas|
|        Brazil|South America| 212710692|   2917562|    NULL|      98644|     NULL|       2047660|        NULL|     771258|         

In [2]:
from pyspark.sql.functions import sum,round
result = df.select(round((sum("TotalDeaths")*100.0/sum("totalCases")),2).alias("global_death_percent"))
result.show()

+--------------------+
|global_death_percent|
+--------------------+
|                3.72|
+--------------------+



In [3]:
from pyspark.sql.functions import col, round, when
df_with_percent = df.select(col("Country/Region"),col("TotalCases"),col("TotalDeaths"),
round((col("TotalDeaths")*100.0/when(col("TotalCases")!=0,col("TotalCases"))),2).alias("local_death_percent"))
result = df_with_percent.orderBy(col("local_death_percent").desc())
result.show()

+---------------+----------+-----------+-------------------+
| Country/Region|TotalCases|TotalDeaths|local_death_percent|
+---------------+----------+-----------+-------------------+
|          Yemen|      1768|        508|              28.73|
|         France|    195633|      30312|              15.49|
|             UK|    308134|      46413|              15.06|
|          Italy|    249204|      35187|              14.12|
|        Belgium|     71158|       9859|              13.86|
|        Hungary|      4597|        600|              13.05|
|         Mexico|    462690|      50517|              10.92|
|    Netherlands|     56982|       6153|               10.8|
|   Sint Maarten|       160|         16|               10.0|
| Western Sahara|        10|          1|               10.0|
|           Chad|       942|         76|               8.07|
|          Spain|    354530|      28500|               8.04|
|Channel Islands|       597|         47|               7.87|
|     Montserrat|       

2. To find out the infected population percentage locally and globally

In [4]:
result = df.select(round((sum("TotalCases")*100.0)/sum("Population"),2).alias("global_infection_percent"))
result.show()

+------------------------+
|global_infection_percent|
+------------------------+
|                     0.3|
+------------------------+



In [5]:
result = df.select(col("Country/Region"), col("TotalCases"), col("Population"),
round((col("TotalCases") * 100.0 / when(col("Population") != 0, col("Population"))), 2).alias("local_infection_percent"))
result = result.orderBy(col("local_infection_percent").desc())
result.show()

+--------------+----------+----------+-----------------------+
|Country/Region|TotalCases|Population|local_infection_percent|
+--------------+----------+----------+-----------------------+
|         Qatar|    112092|   2807805|                   3.99|
| French Guiana|      8127|    299385|                   2.71|
|       Bahrain|     42889|   1706669|                   2.51|
|    San Marino|       699|     33938|                   2.06|
|         Chile|    366671|  19132514|                   1.92|
|        Panama|     71418|   4321282|                   1.65|
|        Kuwait|     70045|   4276658|                   1.64|
|          Oman|     80713|   5118446|                   1.58|
|           USA|   5032179| 331198130|                   1.52|
|  Vatican City|        12|       801|                    1.5|
|          Peru|    455409|  33016319|                   1.38|
|        Brazil|   2917562| 212710692|                   1.37|
|       Armenia|     39819|   2963811|                 

3. To find out the countries with the highest infection rates

In [6]:
from pyspark.sql.functions import col,max 
result = df.groupBy("Country/Region").agg(max("TotalCases").alias("most_infected_countries")).orderBy(col("most_infected_countries").desc()).limit(10)
result.show()

+--------------+-----------------------+
|Country/Region|most_infected_countries|
+--------------+-----------------------+
|           USA|                5032179|
|        Brazil|                2917562|
|         India|                2025409|
|        Russia|                 871894|
|  South Africa|                 538184|
|        Mexico|                 462690|
|          Peru|                 455409|
|         Chile|                 366671|
|      Colombia|                 357710|
|         Spain|                 354530|
+--------------+-----------------------+



4. To find out the countries and continents with the highest death counts

In [7]:
from pyspark.sql.functions import col, sum, coalesce, lit
result = df.groupBy("Country/Region").agg(sum(coalesce(col("TotalDeaths"), lit(0))).alias("highest_death_count"))
result = result.orderBy(col("highest_death_count").desc()).limit(10)
result.show()

+--------------+-------------------+
|Country/Region|highest_death_count|
+--------------+-------------------+
|           USA|             162804|
|        Brazil|              98644|
|        Mexico|              50517|
|            UK|              46413|
|         India|              41638|
|         Italy|              35187|
|        France|              30312|
|         Spain|              28500|
|          Peru|              20424|
|          Iran|              17976|
+--------------+-------------------+



5. Average number of deaths by day (Continents and Countries)

In [8]:
from pyspark.sql.functions import avg
result = df.groupBy("Continent").agg(round(avg(coalesce(col("NewDeaths"), lit(0))),2).alias("avg_daily_deaths"))
result = result.orderBy(col("avg_daily_deaths").desc())
result.show()

+-----------------+----------------+
|        Continent|avg_daily_deaths|
+-----------------+----------------+
|    North America|            23.4|
|    South America|            5.71|
|             Asia|            0.02|
|           Europe|             0.0|
|           Africa|             0.0|
|             NULL|             0.0|
|Australia/Oceania|             0.0|
+-----------------+----------------+



6. Average of cases divided by the number of population of each country (TOP
10)

In [9]:
result = df.select(col("Country/Region"), col("TotalCases"), col("Population"),
round((col("TotalCases") * 100.0) / when(col("Population") != 0, col("Population")), 2).alias("infection_percent"))
result = result.orderBy(col("infection_percent").desc()).limit(10)
result.show()

+--------------+----------+----------+-----------------+
|Country/Region|TotalCases|Population|infection_percent|
+--------------+----------+----------+-----------------+
|         Qatar|    112092|   2807805|             3.99|
| French Guiana|      8127|    299385|             2.71|
|       Bahrain|     42889|   1706669|             2.51|
|    San Marino|       699|     33938|             2.06|
|         Chile|    366671|  19132514|             1.92|
|        Panama|     71418|   4321282|             1.65|
|        Kuwait|     70045|   4276658|             1.64|
|          Oman|     80713|   5118446|             1.58|
|           USA|   5032179| 331198130|             1.52|
|  Vatican City|        12|       801|              1.5|
+--------------+----------+----------+-----------------+



Using JOINS to combine the covid_deaths and covid_vaccine tables :
1. To find out the population vs the number of people vaccinated

In [9]:
import pyspark
from pyspark.sql.functions import col,round,max
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MakerPlanJoins").getOrCreate()
df_world = spark.read.csv('worldometer.csv',header=True,inferSchema = True)
df_covid = spark.read.csv('covid.csv',header=True,inferSchema=True)
joined_df = df_world.join(df_covid, on="Country/Region")

result = joined_df.groupBy("Country/Region", "Population") \
	.agg(max("People_vaccinated").alias("total_people_vaccinated")) \
	.withColumn("percent_vaccinated", round(col("total_people_vaccinated") * 100.0 / col("Population"), 2)) \
	.orderBy(col("percent_vaccinated").desc())

result.show()

+--------------+----------+-----------------------+------------------+
|Country/Region|Population|total_people_vaccinated|percent_vaccinated|
+--------------+----------+-----------------------+------------------+
|         Spain|  46756648|               90174564|            192.86|
|     Argentina|  45236884|               73644938|             162.8|
|     Australia|  25528864|               28271812|            110.74|
|  South Africa|  59381566|               60756794|            102.32|
|        Canada|  37775022|               36451668|              96.5|
|  Saudi Arabia|  34865919|               33514620|             96.12|
|        Poland|  37842302|               35984096|             95.09|
|         Japan| 126435859|               85256419|             67.43|
|         Egypt| 102516525|               67109882|             65.46|
|        Turkey|  84428331|               53611439|              63.5|
|            UK|  67922029|               42835856|             63.07|
|     

2. To find out the percentage of different vaccine taken by people in a country

In [17]:
from pyspark.sql.functions import count

# Rename the column to remove the trailing space
joined_df = joined_df.withColumnRenamed("Vaccines ", "Vaccines")

# Filter and group by the corrected column name
joined_df.filter(col("Country/Region") == "India").groupBy("Country/Region", "Vaccines").agg(count("*").alias("record_count")).show()

+--------------+----------+------------+
|Country/Region|  Vaccines|record_count|
+--------------+----------+------------+
|         India| Covaxin  |           1|
+--------------+----------+------------+



3. To find out percentage of people who took both the doses

In [19]:
from pyspark.sql.functions import col, max, round

# Use the correct DataFrame (df_covid) for aggregation
grouped_df = df_covid.groupBy("Country/Region").agg(
    max("People_vaccinated").alias("total_vaccinated"),
    max("People_fully_vaccinated").alias("total_fully_vaccinated")
)

# Calculate the percentage of fully vaccinated people
result_df = grouped_df.withColumn(
    "percent_fully_vaccinated",
    round(col("total_fully_vaccinated") * 100.0 / col("total_vaccinated"), 2)
)

result_df.show()

+--------------+----------------+----------------------+------------------------+
|Country/Region|total_vaccinated|total_fully_vaccinated|percent_fully_vaccinated|
+--------------+----------------+----------------------+------------------------+
|        Russia|        71888907|              67254980|                   93.55|
|        Turkey|        53611439|              34632934|                    64.6|
|       Germany|        45784782|              17216543|                    37.6|
|        France|         2336961|               2010072|                   86.01|
|     Argentina|        73644938|              71103845|                   96.55|
|         India|        32095958|              31420042|                   97.89|
|         China|        33950722|              18860949|                   55.55|
|       Nigeria|        54725869|              31841587|                   58.18|
|         Italy|        37861529|              16696305|                    44.1|
|         Spain|